In [ ]:
# Import necessary modules
import numpy as np
from scipy.io import loadmat
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report
import glob

# === Summarization function ===
def summarize_feature(feat):
    return np.hstack([
        np.mean(feat, axis=1),
        np.std(feat, axis=1),
        np.min(feat, axis=1),
        np.max(feat, axis=1)
    ])

# === Load features and labels ===
feature_files = sorted(glob.glob('features_sample*.mat'))
X = []
y = []

for f in feature_files:
    data = loadmat(f)
    mfcc = data['mfcc']
    centroid = data['centroid']
    rolloff = data['rolloff']
    mel = data['mel']
    chroma = data['chroma']

    feature_vector = np.hstack([
        summarize_feature(mfcc),
        summarize_feature(centroid),
        summarize_feature(rolloff),
        summarize_feature(mel),
        summarize_feature(chroma)
    ])
    X.append(feature_vector)

with open('labels.txt', 'r') as f:
    for line in f:
        y.append(line.strip())

X = np.array(X)
y = np.array(y)

# Encode labels
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Split into train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

# Standard scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Instantiate and train KNN Classifier
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

# Predict and evaluate
y_pred = knn.predict(X_test)
print(classification_report(y_test, y_pred, target_names=le.classes_))
